# 【課題】量子相関を調べる

In [ ]:
# まずは全てインポート
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

print('notebook ready')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
print(simulator.name())

In [ ]:
circuits = []
for ic in range(4):
    circuit = QuantumCircuit(2, name='circuit{}'.format(ic))
    circuit.h(0)
    circuit.cx(0, 1)
    circuits.append(circuit)

circuits[0].ry(-np.pi / 4., 1)
circuits[1].ry(-3. * np.pi / 4., 1)
circuits[2].ry(-np.pi / 4., 1)
circuits[3].ry(-3. * np.pi / 4., 1)

circuits[2].ry(-np.pi / 2., 0)
circuits[3].ry(-np.pi / 2., 0)

for circuit in circuits:
    circuit.measure_all()
    
shots = 10000
    
sim_job = execute(circuits, backend=simulator, shots=shots)

sim_result = sim_job.result()

C = []
for circuit in circuits:
    c = sim_result.get_counts(circuit)
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)
    
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]
print('S =', S)

In [ ]:
for circuit in circuits:
    ax = plt.figure().add_subplot()
    circuit.draw('mpl', ax=ax)

In [ ]:
# Define theta and phi values
thetas = [0., np.pi / 4., np.pi / 2., 3. * np.pi / 4., np.pi]
theta_labels = ['0', r'$\pi/4$', r'$\pi/2$', r'$3\pi/4$', r'$\pi$']
phis = np.linspace(0., np.pi, 16, endpoint=True)

# Construct a circuit for each (theta, phi) pair
circuits = []
for itheta, theta in enumerate(thetas):
    for iphi, phi in enumerate(phis):
        circuit = QuantumCircuit(2, name='circuit_{}_{}'.format(itheta, iphi))

        ##################
        ### EDIT BELOW ###
        ##################

        #circuit.?
        
        ##################
        ### EDIT ABOVE ###
        ##################
        
        circuit.measure_all()

        circuits.append(circuit)

# Execute the circuit in qasm_simulator and retrieve the results
simulator = Aer.get_backend('qasm_simulator')
shots = 10000
sim_job = execute(circuits, backend=simulator, shots=shots)
result = sim_job.result()

# Plot C versus (theta - phi) for each theta
icirc = 0
for itheta, theta in enumerate(thetas):
    x = theta - phis
    y = np.zeros_like(x)

    for iphi, phi in enumerate(phis):
        c = result.get_counts(circuits[icirc])

        ##################
        ### EDIT BELOW ###
        ##################

        #y[iphi] = ?
        
        ##################
        ### EDIT ABOVE ###
        ##################

        icirc += 1

    plt.plot(x, y, 'o', label=theta_labels[itheta])

plt.legend()
plt.xlabel(r'$\theta - \phi$')
plt.ylabel(r'$\langle \Psi | \sigma^{\phi}_B \sigma^{\theta}_A | \Psi \rangle$')

In [ ]:
def classical_S(params):
    """
    Four-parameter function to be minimized. Returns -|S| = -|<sigma^k sigma^l> - <sigma^k sigma^n> + <sigma^m sigma^l> + <sigma^m sigma^n>|.
    
    Args:
        params (np.ndarray): Values of kappa, lambda, mu, nu
        
    Returns:
        float: -|S|
    """
    
    k, l, m, n = params
    
    S = 0.
    
    # S from |00>
    ##################
    ### EDIT BELOW ###
    ##################
    #sksl_00 = 
    #sksn_00 = 
    #smsl_00 = 
    #smsn_00 = 
    #S += (sksl_00 - sksn_00 + smsl_00 + smsn_00) * 0.5
    ##################
    ### EDIT ABOVE ###
    ##################
    
    # S from |11>
    ##################
    ### EDIT BELOW ###
    ##################
    #sksl_11 = 
    #sksn_11 = 
    #smsl_11 = 
    #smsn_11 = 
    #S += (sksl_11 - sksn_11 + smsl_11 + smsn_11) * 0.5
    ##################
    ### EDIT ABOVE ###
    ##################
    
    return -np.abs(S)

# Initial values
x0 = np.array([0., np.pi / 2., 0., np.pi / 2.])
# Bounds on all parameters ([0, pi])
bounds = Bounds([0.] * 4, [np.pi] * 4)
# Minimize using scipy.optimize.minimize
res = minimize(classical_S, x0, method='trust-constr', bounds=bounds)
print('argmax |S|: kappa={:.3f}pi, lambda={:.3f}pi, mu={:.3f}pi, nu={:.3f}pi'.format(*(res.x / np.pi)))
print('max |S|:', -res.fun)

In [ ]:
# Construct a circuit for each (theta, phi) pair
circuits = []
for itheta, theta in enumerate(thetas):
    for iphi, phi in enumerate(phis):
        circuit_00 = QuantumCircuit(2, name='circuit_{}_{}_00'.format(itheta, iphi))
        circuit_11 = QuantumCircuit(2, name='circuit_{}_{}_11'.format(itheta, iphi))

        ##################
        ### EDIT BELOW ###
        ##################

        #circuit_00.?
        #circuit_11.?
        
        ##################
        ### EDIT ABOVE ###
        ##################
        
        circuit_00.measure_all()
        circuit_11.measure_all()

        circuits.append(circuit_00)
        circuits.append(circuit_11)

# Execute the circuit in qasm_simulator and retrieve the results
shots = 10000
sim_job = execute(circuits, backend=simulator, shots=shots)
result = sim_job.result()

# Plot C versus (theta - phi) for each theta
icirc = 0
for itheta, theta in enumerate(thetas):
    x = theta - phis
    y = np.zeros_like(x)

    for iphi, phi in enumerate(phis):
        c_00 = result.get_counts(circuits[icirc])
        c_11 = result.get_counts(circuits[icirc + 1])

        ##################
        ### EDIT BELOW ###
        ##################

        #y[iphi] = ?
        
        ##################
        ### EDIT ABOVE ###
        ##################

        icirc += 2

    plt.plot(x, y, 'o', label=theta_labels[itheta])

plt.legend()
plt.xlabel(r'$\theta - \phi$')
plt.ylabel(r'$\langle \Psi | \sigma^{\theta}_A \sigma^{\phi}_B | \Psi \rangle$')

**提出するもの**

- 完成した回路のコード（EDIT BELOW / EDIT ABOVEの間を埋める）とシミュレーション結果によるプロット
- ベル状態と可分状態の混合状態とでの2体相関の違いに関する考察
- （おまけ）可分状態の混合状態で、$|S|=2$を実現するものを考案し、その$\sigma^{\theta}\sigma^{\phi}$の期待値のプロット